<a href="https://colab.research.google.com/github/hemsush/chatgpt_prompts_exercises/blob/main/CustomDataQAChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**QA bot on enterprise data**

**step 1 : Load Data from pdf**

In [1]:
!pip install requests pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 21.3 MB/s eta 0:00:00


In [27]:
import requests
import fitz  # PyMuPDF



In [28]:
# Function to download the PDF from URL
def download_pdf(url, file_name):
    response = requests.get(url)
    response.raise_for_status()  # Check for request success
    with open(file_name, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded PDF to {file_name}")

# Function to extract text from a PDF
def extract_pdf_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""

    # Iterate through each page and extract text
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text()

    return text

# Function to split document into manageable chunks for in-context learning
def split_text_into_sections(text, max_length=1500):
    sections = []
    lines = text.split("\n")
    current_section = ""

    for line in lines:
        if len(current_section) + len(line) > max_length:
            sections.append(current_section)
            current_section = line
        else:
            current_section += " " + line

    if current_section:
        sections.append(current_section)

    return sections


**Step 2 : configure chat completion api from open ai**

In [3]:
!pip install openai

In [4]:
from openai import OpenAI

In [5]:
from google.colab import userdata
# Initialize the OpenAI client
client = OpenAI(
    api_key=userdata.get('open_api_key')
)

**Step 3 : Incontext learning**

In [33]:
# Function to interact with GPT using in-context learning
def chat_gpt_with_incontext_learning(prompt, context):
    # Construct prompt with context and question
    in_context_prompt = f"Document context:\n{context}\n\nQuestion: {prompt}\nAnswer:"

    # Requesting the response from the model
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # You can replace this with another model such as "gpt-4-1106-preview"
        messages=[
            {"role": "system", "content": "You answer questions based on the provided document context."},
            {"role": "user", "content": in_context_prompt}
        ],
        temperature=0.7,  # You can adjust this for creativity
        max_tokens=500  # Adjust as needed for the length of response
    )

    # Correctly accessing the message content as per the new format
    response_message = response.choices[0].message.content
    return response_message.strip()

# Function to get the most relevant response based on the question and document
def get_relevant_response(question, document_text):
    # Split document into smaller sections for better context handling
    sections = split_text_into_sections(document_text)

    # Select the most relevant section based on the question
    context = sections[0]  # For now, we're just using the first section

    # Get response from the model using the in-context learning function
    return chat_gpt_with_incontext_learning(question, context)


In [31]:
# URL of the PDF document
pdf_url = "https://mf.nipponindiaim.com/FundsAndPerformance/ProductNotes/NipponIndia-Short-Term-Fund-Jan-2024.pdf"
pdf_file = "NipponIndia-Short-Term-Fund-Jan-2024.pdf"

# Download the PDF
download_pdf(pdf_url, pdf_file)

# Extract text from the downloaded PDF
pdf_text = extract_pdf_text(pdf_file)

Downloaded PDF to NipponIndia-Short-Term-Fund-Jan-2024.pdf


In [34]:
# Sample Question
question = "What is the investment objective of the Nippon India Short Term Fund?"

# Get response from OpenAI using in-context learning based on the document content
answer = get_relevant_response(question, pdf_text)

print("Answer:", answer)

Answer: The investment objective of the Nippon India Short Term Fund is to generate stable returns for investors with a short-term investment horizon by investing in Debt & Money Market Instruments.
